In [1]:
from sortedcontainers import SortedSet
from utils import *
import cProfile
import pstats
import time
import warnings
import json
import copy

# taken from https://github.com/grantjenks/python-sortedcontainers/issues/53
class FasterSortedSet(SortedSet):
    def union(self, *iterables):
        if (self and len(iterables) == 1
                and isinstance(iterables[0], SortedSet)
                and iterables[0]
                and self._list._lists[-1][-1] < iterables[0]._list._lists[0][0]):
            that = iterables[0]
            result = FasterSortedSet()
            _set = result._set
            _set.update(self)
            _set.update(that)
            _list = result._list
            _list._lists.extend(list(sublist) for sublist in self._list._lists)
            _list._maxes.extend(self._list._maxes)
            _list._lists.extend(list(sublist) for sublist in that._list._lists)
            _list._maxes.extend(that._list._maxes)
            _list._len = len(_set)
            return result
        else:
            return self.__class__(
                chain(iter(self), *iterables),
                key=self._key,
                load=self._load
            )

    __or__ = union
    __ror__ = __or__

In [2]:
import Cython
%load_ext Cython

In [3]:
%%cython
cimport cython
import numpy as np
cimport numpy as np


@cython.boundscheck(False)
@cython.wraparound(False)
def superimpose_potential(np.float32_t[:,:,:,:] pot_field, np.float64_t[:] center, unsigned long scale, double radius, np.float32_t[:,:,:,:] template, unsigned long diameter):
    cdef int[:] scaled_center = np.array([0,0,0], dtype=np.int32)
    scaled_center[0] = np.round(center[0] * scale).astype(np.int32) % pot_field.shape[0]
    scaled_center[1] = np.round(center[1] * scale).astype(np.int32) % pot_field.shape[1]
    scaled_center[2] = np.round(center[2] * scale).astype(np.int32) % pot_field.shape[2]
    #print("Scaled center {}".format(scaled_center))
    #Vmin is at r=2^(1/6)*zero_pot
    cdef np.float32_t zero_pot = radius / 2**(1/5) * scale # in indices
    cdef int low = -((diameter-1)//2)
    cdef int high = (diameter+1)//2
    cdef Py_ssize_t x = (pot_field.shape[0])
    cdef Py_ssize_t y = (pot_field.shape[1])
    cdef Py_ssize_t z = (pot_field.shape[2])
    
    cdef np.float32_t[:,:,:,:] pot_view = pot_field
    cdef np.float32_t[:,:,:,:] tmp_view = template
    
    cdef Py_ssize_t i, ii, j, jj, k, kk
    for i in range(low, high):
        ii = (scaled_center[0] + i) % x
        idist = (i)**2
        for j in range(low, high):
            jj = (scaled_center[1] + j) % y
            jdist = (j)**2
            for k in range(low, high):
                kk = scaled_center[2] + k
                if kk < 0 or kk > z-1:
                    continue
                if i == 0 and j == 0 and k == 0:
                    pot_view[ii,jj,kk,0] = 0
                    pot_view[ii,jj,kk,1] = 0
                    pot_view[ii,jj,kk,2] = 0
                    pot_view[ii,jj,kk,3] = np.inf
                
                #r = np.sqrt(idist + jdist + (k)**2)
                #magnitude = tmp_view[i-low,j-low,k-low,3]#(zero_pot/r)**12 - (zero_pot/r)**6
                #direction = tmp_view[i-low,j-low,k-low,:3]#np.array([i,j,k]) / distance3D([i,j,k], [0,0,0]) * magnitude/np.abs(magnitude) # get direction
                #magnitude = np.abs(magnitude) # set magnitude to the absolute value
                
                pot_view[ii,jj,kk,0] = pot_view[ii,jj,kk,3]*pot_view[ii,jj,kk,0] + tmp_view[i-low,j-low,k-low,3]*tmp_view[i-low,j-low,k-low,0]
                pot_view[ii,jj,kk,1] = pot_view[ii,jj,kk,3]*pot_view[ii,jj,kk,1] + tmp_view[i-low,j-low,k-low,3]*tmp_view[i-low,j-low,k-low,1]
                pot_view[ii,jj,kk,2] = pot_view[ii,jj,kk,3]*pot_view[ii,jj,kk,2] + tmp_view[i-low,j-low,k-low,3]*tmp_view[i-low,j-low,k-low,2]
                #pot_view[ii,jj,kk,3] = np.sqrt(pot_view[ii,jj,kk,0]**2 + pot_view[ii,jj,kk,1]**2 + pot_view[ii,jj,kk,2]**2)
                pot_view[ii,jj,kk,3] = pot_view[ii,jj,kk,3] + tmp_view[i-low, j-low, k-low, 3]
                if pot_view[ii,jj,kk,3] == 0.0:
                    pot_view[ii,jj,kk,0] = 0.0
                    pot_view[ii,jj,kk,1] = 0.0
                    pot_view[ii,jj,kk,2] = 0.0
                else:
                    pot_view[ii,jj,kk,0] = pot_view[ii,jj,kk,0]/pot_view[ii,jj,kk,3]
                    pot_view[ii,jj,kk,1] = pot_view[ii,jj,kk,1]/pot_view[ii,jj,kk,3]
                    pot_view[ii,jj,kk,2] = pot_view[ii,jj,kk,2]/pot_view[ii,jj,kk,3]
#     ii = scaled_center[0] + low
#     jj = scaled_center[1] + low
#     kk = scaled_center[2] + low
#     vectors = [0, 1, 2]
#     if scaled_center[0] > -low and scaled_center[0] + high < x-1:
#         if scaled_center[1] > -low and scaled_center[1] + high < y-1:
#             if scaled_center[2] > -low and scaled_center[2] + high < z-1:
#                 # easy case: entire added field doesn't cross boundaries
#                 region = pot_field[ii:ii+diameter,jj:jj+diameter,kk:kk+diameter,:]
#                 print(region.shape, template.shape)
#                 region[:,:,:,:3] = region[:,:,:,3]*region[:,:,:,:3] + template[:,:,:,3]*template[:,:,:,:3]
#                 region[:,:,:,3] = np.sqrt(region[:,:,:,0]**2 + region[:,:,:,1]**2 + region[:,:,:,2]**2)
#                 region = np.where(region[:,:,:,3]==0.0, [0,0,0,0], [region[:,:,:,:3]/region[:,:,:,3], region[:,:,:,3]])
#             elif scaled_center[2] <= -low:
#                 # 0<x<L, 0<y<L, 0=z<h
#                 region = pot_field[ii:ii+diameter,jj:jj+diameter,0:kk+diameter,:]
#                 t = template[:,:,-kk-diameter:,:]
#                 print(scaled_center, region.shape, template[:,:,-kk-diameter:,:].shape, low, kk)
#                 cs = [region[:,:,:,3]*region[:,:,:,i] + t[:,:,:,3]*t[:,:,:,i] for i in vectors]
#                 region[:,:,:,:3] = np.stack(cs, axis=3)
#                 region[:,:,:,3] = np.sqrt(region[:,:,:,0]**2 + region[:,:,:,1]**2 + region[:,:,:,2]**2)
#                 region = np.stack([region[:,:,:,0]/region[:,:,:,3], region[:,:,:,1]/region[:,:,:,3], region[:,:,:,2]/region[:,:,:,3], region[:,:,:,3]], axis=3)
#                 region = np.where(region==np.nan, 0, region)
#             else:
#                 # 0<x<L, 0<y<L, 0<z=h
#                 region = pot_field[ii:ii+diameter,jj:jj+diameter,kk:-1,:]
#                 region[:,:,:,:3] = region[:,:,:,3]*region[:,:,:,:3] + template[:,:,:low,3]*template[:,:,:low,:3]
#                 region[:,:,:,3] = np.sqrt(region[:,:,:,0]**2 + region[:,:,:,1]**2 + region[:,:,:,2]**2)
#                 region = np.where(region[:,:,:,3]==0.0, [0,0,0,0], [region[:,:,:,:3]/region[:,:,:,3], region[:,:,:,3]])
#         elif scaled_center[0] <= -low:
#             if scaled_center[2] > -low and scaled_center[2] + high < z-1:
#                 # 0<x<L, 0=y<L, 0<z<h
#                 region = pot_field[ii:ii+diameter,0:jj+diameter,kk:kk+diameter,:]
#                 region[:,:,:,:3] = region[:,:,:,3]*region[:,:,:,:3] + template[:,-low:,:,3]*template[:,-low:,:,:3]
#                 region[:,:,:,3] = np.sqrt(region[:,:,:,0]**2 + region[:,:,:,1]**2 + region[:,:,:,2]**2)
#                 region = np.where(region[:,:,:,3]==0.0, [0,0,0,0], [region[:,:,:,:3]/region[:,:,:,3], region[:,:,:,3]])
                
#                 #0<x<L, y<0, 0<z<h
#                 region = pot_field[ii:ii+diameter,low:,kk:kk+diameter,:]
#                 region[:,:,:,:3] = region[:,:,:,3]*region[:,:,:,:3] + template[:,:low,:,3]*template[:,:low,:,:3]
#                 region[:,:,:,3] = np.sqrt(region[:,:,:,0]**2 + region[:,:,:,1]**2 + region[:,:,:,2]**2)
#                 region = np.where(region[:,:,:,3]==0.0, [0,0,0,0], [region[:,:,:,:3]/region[:,:,:,3], region[:,:,:,3]])
#             elif scaled_center[2] <= -low:
#                 # 0<x<L, 0=y<L, 0=z<h
#                 region = pot_field[ii:ii+diameter,0:jj+diameter,0:kk+diameter,:]
#                 region[:,:,:,:3] = region[:,:,:,3]*region[:,:,:,:3] + template[:,-low:,-low:,3]*template[:,-low:,-low:,:3]
#                 region[:,:,:,3] = np.sqrt(region[:,:,:,0]**2 + region[:,:,:,1]**2 + region[:,:,:,2]**2)
#                 region = np.where(region[:,:,:,3]==0.0, [0,0,0,0], [region[:,:,:,:3]/region[:,:,:,3], region[:,:,:,3]])
                
#                 #0<x<L, y<0, 0=z<h
#                 region = pot_field[ii:ii+diameter,low:,0:kk+diameter,:]
#                 region[:,:,:,:3] = region[:,:,:,3]*region[:,:,:,:3] + template[:,:low,:low,3]*template[:,:low,:low,:3]
#                 region[:,:,:,3] = np.sqrt(region[:,:,:,0]**2 + region[:,:,:,1]**2 + region[:,:,:,2]**2)
#                 region = np.where(region[:,:,:,3]==0.0, [0,0,0,0], [region[:,:,:,:3]/region[:,:,:,3], region[:,:,:,3]])
#             else:
#                 # 0<x<L, 0=y<L, 0<z=h
#                 region = pot_field[ii:ii+diameter,0:jj+diameter,kk:-1,:]
#                 region[:,:,:,:3] = region[:,:,:,3]*region[:,:,:,:3] + template[:,-low:,:low,3]*template[:,-low:,:low,:3]
#                 region[:,:,:,3] = np.sqrt(region[:,:,:,0]**2 + region[:,:,:,1]**2 + region[:,:,:,2]**2)
#                 region = np.where(region[:,:,:,3]==0.0, [0,0,0,0], [region[:,:,:,:3]/region[:,:,:,3], region[:,:,:,3]])
                
#                 # 0<x<L, y<0, 0<z=h
#                 region = pot_field[ii:ii+diameter,low:,kk:-1,:]
#                 region[:,:,:,:3] = region[:,:,:,3]*region[:,:,:,:3] + template[:,:low,:low,3]*template[:,:low,:low,:3]
#                 region[:,:,:,3] = np.sqrt(region[:,:,:,0]**2 + region[:,:,:,1]**2 + region[:,:,:,2]**2)
#                 region = np.where(region[:,:,:,3]==0.0, [0,0,0,0], [region[:,:,:,:3]/region[:,:,:,3], region[:,:,:,3]])
#         else:
#             if scaled_center[2] > -low and scaled_center[2] + high < z-1:
#                 # 0<x<L, 0<y=L, 0<z<h
#                 region = pot_field[ii:ii+diameter,jj:,kk:kk+diameter,:]
#                 region[:,:,:,:3] = region[:,:,:,3]*region[:,:,:,:3] + template[:,low:,:,3]*template[:,low:,:,:3]
#                 region[:,:,:,3] = np.sqrt(region[:,:,:,0]**2 + region[:,:,:,1]**2 + region[:,:,:,2]**2)
#                 region = np.where(region[:,:,:,3]==0.0, [0,0,0,0], [region[:,:,:,:3]/region[:,:,:,3], region[:,:,:,3]])
                
#                 #0<x<L, y>L, 0<z<h
#                 region = pot_field[ii:ii+diameter,low:,kk:kk+diameter,:]
#                 region[:,:,:,:3] = region[:,:,:,3]*region[:,:,:,:3] + template[:,:low,:,3]*template[:,:low,:,:3]
#                 region[:,:,:,3] = np.sqrt(region[:,:,:,0]**2 + region[:,:,:,1]**2 + region[:,:,:,2]**2)
#                 region = np.where(region[:,:,:,3]==0.0, [0,0,0,0], [region[:,:,:,:3]/region[:,:,:,3], region[:,:,:,3]])
#             elif scaled_center[2] <= -low:
#                 # 0<x<L, 0=y<L, 0=z<h
#                 region = pot_field[ii:ii+diameter,0:jj+diameter,0:kk+diameter,:]
#                 region[:,:,:,:3] = region[:,:,:,3]*region[:,:,:,:3] + template[:,-low:,-low:,3]*template[:,-low:,-low:,:3]
#                 region[:,:,:,3] = np.sqrt(region[:,:,:,0]**2 + region[:,:,:,1]**2 + region[:,:,:,2]**2)
#                 region = np.where(region[:,:,:,3]==0.0, [0,0,0,0], [region[:,:,:,:3]/region[:,:,:,3], region[:,:,:,3]])
                
#                 #0<x<L, y<0, 0=z<h
#                 region = pot_field[ii:ii+diameter,low:,0:kk+diameter,:]
#                 region[:,:,:,:3] = region[:,:,:,3]*region[:,:,:,:3] + template[:,:low,:low,3]*template[:,:low,:low,:3]
#                 region[:,:,:,3] = np.sqrt(region[:,:,:,0]**2 + region[:,:,:,1]**2 + region[:,:,:,2]**2)
#                 region = np.where(region[:,:,:,3]==0.0, [0,0,0,0], [region[:,:,:,:3]/region[:,:,:,3], region[:,:,:,3]])
#             else:
#                 # 0<x<L, 0=y<L, 0<z=h
#                 region = pot_field[ii:ii+diameter,0:jj+diameter,kk:-1,:]
#                 region[:,:,:,:3] = region[:,:,:,3]*region[:,:,:,:3] + template[:,-low:,:low,3]*template[:,-low:,:low,:3]
#                 region[:,:,:,3] = np.sqrt(region[:,:,:,0]**2 + region[:,:,:,1]**2 + region[:,:,:,2]**2)
#                 region = np.where(region[:,:,:,3]==0.0, [0,0,0,0], [region[:,:,:,:3]/region[:,:,:,3], region[:,:,:,3]])
                
#                 # 0<x<L, y<0, 0<z=h
#                 region = pot_field[ii:ii+diameter,low:,kk:-1,:]
#                 region[:,:,:,:3] = region[:,:,:,3]*region[:,:,:,:3] + template[:,:low,:low,3]*template[:,:low,:low,:3]
#                 region[:,:,:,3] = np.sqrt(region[:,:,:,0]**2 + region[:,:,:,1]**2 + region[:,:,:,2]**2)
#                 region = np.where(region[:,:,:,3]==0.0, [0,0,0,0], [region[:,:,:,:3]/region[:,:,:,3], region[:,:,:,3]])
    return

@cython.boundscheck(False)
@cython.wraparound(False)
def shortest_distance_line_to_point(double[:] line_1, double[:] line_2, np.float32_t[:] point):
    x = np.array([line_1[0] - line_2[0], line_1[1] - line_2[1], line_1[2] - line_2[2]])
    pmq = np.array([point[0] - line_2[0], point[1] - line_2[1], point[2] - line_2[2]])
    t = np.dot(pmq, x)/np.dot(x, x)
    r = np.linalg.norm(t*x - pmq)
    return r

In [4]:
def distance3D(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)

def distance3DPBC(p1, p2, L):
    # distance in x considering PBC
    if (p1[0] - p2[0]) > L/2:
        dx = p1[0] - p2[0] - L
    elif p1[0] - p2[0] < -L/2:
        dx = p1[0] - p2[0] + L
    else:
        dx = p1[0] - p2[0]
    # distance in y considering PBC
    if (p1[1] - p2[1]) > L/2:
        dy = p1[1] - p2[1] - L
    elif p1[1] - p2[1] < -L/2:
        dy = p1[1] - p2[1] + L
    else:
        dy = p1[1] - p2[1]
    return np.sqrt(dx**2 + dy**2 + (p1[2] - p2[2])**2)

# returns distance^2 (for faster comparisons)
def distance3D2PBC(p1, p2, L):
    # distance in x considering PBC
    if (p1[0] - p2[0]) > L/2:
        dx = p1[0] - p2[0] - L
    elif p1[0] - p2[0] < -L/2:
        dx = p1[0] - p2[0] + L
    else:
        dx = p1[0] - p2[0]
    # distance in y considering PBC
    if (p1[1] - p2[1]) > L/2:
        dy = p1[1] - p2[1] - L
    elif p1[1] - p2[1] < -L/2:
        dy = p1[1] - p2[1] + L
    else:
        dy = p1[1] - p2[1]
    return dx**2 + dy**2 + (p1[2] - p2[2])**2

def distance2D2PBC(p1, p2, L):
    # distance in x considering PBC
    if (p1[0] - p2[0]) > L/2:
        dx = p1[0] - p2[0] - L
    elif p1[0] - p2[0] < -L/2:
        dx = p1[0] - p2[0] + L
    else:
        dx = p1[0] - p2[0]
    # distance in y considering PBC
    if (p1[1] - p2[1]) > L/2:
        dy = p1[1] - p2[1] - L
    elif p1[1] - p2[1] < -L/2:
        dy = p1[1] - p2[1] + L
    else:
        dy = p1[1] - p2[1]
    return dx**2 + dy**2

# p is the current point position, n the unit vector, and step_size the magnitude
def step_along_line(p, n, step_size):
    return p + n * step_size

In [5]:
class SlantedCorridors:
    def __init__(self, length, bin_size, theta, height, verbose=False, *args, **kwargs):
        self.length = length
        self.theta = theta
        self.tantheta = tand(theta)
        self.cos1theta = cosd(theta)
        self.sin1theta = sind(theta)
        self.height = height
        self.bin_size = bin_size
        self.bins_side = int(length / bin_size)
        
        self.bin_nums = int(self.bins_side**2)
        self.bins = []
        #self.bins = [[] for _ in range(self.bin_nums)]
        #for b in self.bins:
        #    b = SortedSet(key=lambda position: position[1])
        #self.bins = [FasterSortedSet(key=lambda position: position[1])] * self.bin_nums
        for i in range(self.bin_nums):
            self.bins.append(FasterSortedSet(key=lambda position: position[1]))
        print("Total number of bins: {}".format(len(self.bins)))
        
        self.verbose = verbose
        
    def find_bin(self, position):
        try:
            adjusted_x = position[2] * self.tantheta + position[0]
            spacex = int(np.floor((adjusted_x % self.length)//self.bin_size))
            spacey = int(np.floor((position[1] % self.length)//self.bin_size))
        except FloatingPointError:
            raise FloatingPointError("\n*\tposition: {}\n*\tself.tantheta: {}\n*\tself.length: {}\n*\tself.bin_size: {}".format(position, self.tantheta, self.length, self.bin_size))
        
        idx = spacex + spacey*self.bins_side
        #print(position % (self.length, self.length, np.inf), idx, spacex, spacey, spacez, bin_size)
        
        return self.bins[idx]
    
    # over-inclusive as it assumes a square with side length 2x radius
    def add_to_bins(self, center, radius, idx):
        bins = self.find_bins(center, radius)
        priority = self.calc_priority(center)
        for b in bins:
            b.add((idx, priority))
        return bins
    
    # over-inclusive as it assumes a cube with side length 2x radius
    def find_bins(self, center, radius):
        center_bin = self.find_bin(center)
        temp = [center_bin]
        
        left_back_lower_bin = self.find_bin((center + np.array(([-radius, -radius, -radius]))) % [self.length, self.length, np.inf])
        if left_back_lower_bin not in temp:
            temp.append(left_back_lower_bin)
            
        left_front_lower_bin = self.find_bin((center + np.array([-radius, radius, -radius])) % [self.length, self.length, np.inf])
        if left_front_lower_bin not in temp:
            temp.append(left_front_lower_bin)
        
        left_back_upper_bin = self.find_bin((center + np.array([-radius, -radius, radius])) % [self.length, self.length, np.inf])
        if left_back_upper_bin not in temp:
            temp.append(left_back_upper_bin)
            
        left_front_upper_bin = self.find_bin((center + np.array([-radius, radius, radius])) % [self.length, self.length, np.inf])
        if left_front_upper_bin not in temp:
            temp.append(left_front_upper_bin)
            
        right_back_lower_bin = self.find_bin((center + np.array([radius, -radius, -radius])) % [self.length, self.length, np.inf])
        if right_back_lower_bin not in temp:
            temp.append(right_back_lower_bin)
            
        right_front_lower_bin = self.find_bin((center + np.array([radius, radius, -radius])) % [self.length, self.length, np.inf])
        if right_front_lower_bin not in temp:
            temp.append(right_front_lower_bin)
            
        right_back_upper_bin = self.find_bin((center + np.array([radius, -radius, radius])) % [self.length, self.length, np.inf])
        if right_back_upper_bin not in temp:
            temp.append(right_back_upper_bin)
            
        right_front_upper_bin = self.find_bin((center + np.array([radius, radius, radius])) % [self.length, self.length, np.inf])
        if right_front_upper_bin not in temp:
            temp.append(right_front_upper_bin)
        
        return temp
    
    def find_bin_idx(self, position):
        adjusted_x = position[2] * self.tantheta + position[0]
        spacex = int(np.floor((adjusted_x % self.length)//self.bin_size))
        spacey = int(np.floor((position[1] % self.length)//self.bin_size))
        
        idx = spacex + spacey*self.bins_side
        if self.verbose:
            print("Position: {}".format(position))
            #print("self.tandtheta: {}, {}".format(self.tantheta, tand(self.theta)))
            print("adjusted_x: {}".format(adjusted_x))
            print("Indices: {}, {}, {}".format(spacex, spacey, idx))
        return idx
    
    # over-inclusive as it assumes a cube with side length 2x radius
    def find_bins_idx(self, center, radius):
        center_bin = self.find_bin_idx(center)
        temp = [center_bin]
        
        left_back_lower_bin = self.find_bin_idx((center + np.array(([-radius, -radius, -radius]))) % [self.length, self.length, self.height])
        if left_back_lower_bin not in temp:
            temp.append(left_back_lower_bin)
            
        left_front_lower_bin = self.find_bin_idx((center + np.array([-radius, radius, -radius])) % [self.length, self.length, self.height])
        if left_front_lower_bin not in temp:
            temp.append(left_front_lower_bin)
        
        left_back_upper_bin = self.find_bin_idx((center + np.array([-radius, -radius, radius])) % [self.length, self.length, self.height])
        if left_back_upper_bin not in temp:
            temp.append(left_back_upper_bin)
            
        left_front_upper_bin = self.find_bin_idx((center + np.array([-radius, radius, radius])) % [self.length, self.length, self.height])
        if left_front_upper_bin not in temp:
            temp.append(left_front_upper_bin)
            
        right_back_lower_bin = self.find_bin_idx((center + np.array([radius, -radius, -radius])) % [self.length, self.length, self.height])
        if right_back_lower_bin not in temp:
            temp.append(right_back_lower_bin)
            
        right_front_lower_bin = self.find_bin_idx((center + np.array([radius, radius, -radius])) % [self.length, self.length, self.height])
        if right_front_lower_bin not in temp:
            temp.append(right_front_lower_bin)
            
        right_back_upper_bin = self.find_bin_idx((center + np.array([radius, -radius, radius])) % [self.length, self.length, self.height])
        if right_back_upper_bin not in temp:
            temp.append(right_back_upper_bin)
            
        right_front_upper_bin = self.find_bin_idx(center + np.array([radius, radius, radius]))
        if right_front_upper_bin not in temp:
            temp.append(right_front_upper_bin)
            
        return temp
    
    # return particles that share bins
    def get_nearby_particles(self, center, radius):
        bins = self.find_bins(center, radius)
        
        if bins is []:
            return [] # should never happen
        
        temp = []
        for b in bins:
            if len(b) > 0:
                for i in b:
                    if i not in temp:
                        temp.append(i)
        return temp
    
    def drop_particle(self, center, radius, atoms, profile=False):
        if profile:
            pr = cProfile.Profile()
            pr.enable()
        bins = FasterSortedSet(key=lambda position: position[1])
        # combine the bins into one larger bin
        fbins = self.find_bins(center, radius)
        for b in fbins:
            bins |= b
        adjusted_x = center[2] * self.tantheta + center[0]
        factor = adjusted_x // self.length
        adjusted_x %= self.length
        center_float = center.astype(float)
        
        
        for i in range(0, len(bins)):
            # check collision with bins[i][0]-th particle
            particle = atoms[bins[len(bins) - i-1][0],:]
            rs = radius + particle[4]
                
            x_on_particle_z = (center[2] - particle[2]) * self.tantheta + center[0] # x coordinate on z=particle[2] plane
            factor = x_on_particle_z // self.length                   # get multiple of self.length to shift by
            x_on_particle_z %= self.length                            # restrict x coordinate within PBC
            
            #center_proj_sub = np.array([x_on_particle_z, center[1], particle[2]])
            #center_proj_height = center_float - [factor*self.length, 0, 0]
            
            offset = [0,0,0]
            # offset_x, shift center nearer to account for PBC
            #if particle[0] < rs and center_proj_sub[0] > self.length - rs:
            if particle[0] - x_on_particle_z < -self.length/2:
                offset[0] = -self.length
            #elif particle[0] > self.length - rs and center_proj_sub[0] < rs:
            elif particle[0] - x_on_particle_z > self.length/2:
                offset[0] = self.length
            # offset_y, shift center nearer to account for PBC
            #if particle[1] < rs and center_proj_sub[1] > self.length - rs:
            if particle[1] - center[1] < -self.length/2:
                offset[1] = -self.length
            #elif particle[1] > self.length - rs and center_proj_sub[1] < rs:
            elif particle[1] - center[1] > self.length/2:
                offset[1] = self.length
            
            #proj_x = particle[2] * self.tantheta + center[0]
            #print('\t{}'.format(particle[:3]))
            
            #d = shortest_distance_line_to_point(center_proj_height + offset, center_proj_sub+offset, particle)
            radii2 = rs**2
            
            
                
            # rotate axes by theta degrees about y-axis into new coordinate system so that z' axis is unimportant
            #rotx = (center_proj_height[0] - particle[0] + offset[0])*self.cos1theta + (center_proj_height[2] - particle[2])*self.sin1theta
            #d2 = rotx**2 + (center_proj_height[1] - particle[1] + offset[1])**2
            rotx = (center_float[0] - factor*self.length - particle[0] + offset[0])*self.cos1theta + (center_float[2] - particle[2])*self.sin1theta
            d2 = rotx**2 + (center_float[1] - particle[1] + offset[1])**2
            
            if self.verbose:
                print("Generated position: {}".format(center))
                print("x-coordinate in new coordinate system: {}".format(rotx))
                print("Offset: {}".format(offset))
                #print("center_proj_height + offset: {}".format(center_proj_height+offset))
                #print("center_proj_sub + offset: {}".format(center_proj_sub+offset))
                #print("Adjusted generated position: {}".format(center_proj_height))
                print("Existing particle position being considered: {}".format(particle[:3]))
                #print("Offset (if any): {}".format(offset))
                print("Minimum distance^2 between: {}".format(d2))
                print("Radii added together: {}".format(rs))
                
            if radii2 > d2:
                # it collides, calculate the new position
                rotz = np.sqrt(radii2 - d2)
                z = rotx*self.sin1theta + rotz*self.cos1theta + particle[2]
                x = rotx*self.cos1theta - rotz*self.sin1theta + particle[0]

                if z < radius:
                    # if it would collide with the surface below the substrate for this particle then we can skip this particle
                    continue
                if self.verbose:
                    print("Particles collide.")
                    print("Coordinates in new coordinate system: {}".format([rotx, center[1], rotz]))
                    print("Distance: {}".format(np.sqrt((x - particle[0])**2 + (center[1] - particle[1])**2 + (z - particle[2])**2)))
#                     print("Start: {}".format(center_proj_height))
#                     print("Start (reprojected): {}".format(center_proj_height + offset))
#                     print("Target: {}".format(center_proj_sub))
#                     print("Target (reprojected): {}".format(center_proj_sub+offset))
                    print("Collision position: {}".format([x, center[1], z]))
                    print("Colliding atom position: {}\n".format(particle[:3]))
                #print("Hit particle: {} -> {}".format(center, [x, center[1], z]))
                if profile:
                    pr.disable()
                    p = pstats.Stats(pr)
                    p.sort_stats('time').print_stats()
                    #pr.print_stats()
                return np.array([x, center[1], z]), bins[len(bins) - i-1][0]
        x = adjusted_x - radius*self.tantheta
        #print("Didn't hit particle: {} -> {}".format(center, [x, center[1], radius]))
        if self.verbose:
            print("Particle did not collide with existing surface; landing at {}.\n".format([x, center[1], radius]))
        if profile:
            pr.disable()
            pr.print_stats()
        return np.array([x, center[1], radius]), -1
    '''
    z
    z
    z
    z
    z
    z
    z
    z
    z
    z
    z
    z
    z
    z
    z
    z
    z
    z
    z
    '''
    def drop_particle_waffle(self, center, radius, atoms, profile=False):
        if profile:
            pr = cProfile.Profile()
            pr.enable()
        #bins = FasterSortedSet(key=lambda position: position[1])
        # combine the bins into one larger bin
        fbins = self.find_bins(center, radius)
        #for b in fbins:
        #    bins |= b
        bin_len = 0
        val_array = np.zeros(len(fbins))
        particle_array = [None]*len(fbins)
        used_array = [1]*len(fbins)
        for i in range(0, len(fbins)):
            bin_len += len(fbins[i])
            j = 1
            if len(fbins[i]) > 0:
                temp_particle = fbins[i][-j]
                while fbins[i][-j][0] in particle_array and j < len(fbins[i]):
                    j += 1
                    if j > len(fbins[i]):
                        temp_particle = [-np.inf, 0]
                    else:
                        temp_particle = fbins[i][-j]
                particle_array[i] = temp_particle[0]
                val_array[i] = temp_particle[1]
            else:
                particle_array[i] = -np.inf
                val_array[i] = 0
        print("fbins: {}\nparticles: {}\nvalues: {}\nused: {}".format(fbins, particle_array, val_array, used_array))
        used_array[i] = j
            
        adjusted_x = center[2] * self.tantheta + center[0]
        factor = adjusted_x // self.length
        adjusted_x %= self.length
        center_float = center.astype(float)
        
        
        for i in range(0, bin_len):
            print("iteration {}".format(i))
            # figure out which bin has the highest priority particle and replace
            highest_priority_bin = np.argmax(val_array)
            print("\tfbins: {}\n\tparticles: {}\n\tvalues: {}\n\tused: {}".format(fbins, particle_array, val_array, used_array))
            particle = atoms[particle_array[highest_priority_bin], :]
            j = used_array[highest_priority_bin]
            if j < len(fbins[highest_priority_bin]):
                while fbins[highest_priority_bin][-j][0] in particle_array and j < len(fbins[highest_priority_bin]):
                    print("\t\tj: {}".format(j))
                    j += 1
                    if j == len(fbins[highest_priority_bin]):
                        temp_particle = [-np.inf, 0]
                    else:
                        temp_particle = fbins[highest_priority_bin][-j]
                
                particle_array[highest_priority_bin] = temp_particle[0]
                val_array[highest_priority_bin] = temp_particle[1]
            else:
                particle_array[highest_priority_bin] = -np.inf
                val_array[highest_priority_bin] = 0
            used_array[highest_priority_bin] = j
                
            
            # check collision with bins[i][0]-th particle
            #particle = atoms[bins[len(bins) - i-1][0],:]
            rs = radius + particle[4]
            
            x_on_particle_z = (center[2] - particle[2]) * self.tantheta + center[0] # x coordinate on z=particle[2] plane
            factor = x_on_particle_z // self.length                   # get multiple of self.length to shift by
            x_on_particle_z %= self.length                            # restrict x coordinate within PBC
            
            #center_proj_sub = np.array([x_on_particle_z, center[1], particle[2]])
            #center_proj_height = center_float - [factor*self.length, 0, 0]
            
            offset = [0,0,0]
            # offset_x, shift center nearer to account for PBC
            #if particle[0] < rs and center_proj_sub[0] > self.length - rs:
            if particle[0] - x_on_particle_z < -self.length/2:
                offset[0] = -self.length
            #elif particle[0] > self.length - rs and center_proj_sub[0] < rs:
            elif particle[0] - x_on_particle_z > self.length/2:
                offset[0] = self.length
            # offset_y, shift center nearer to account for PBC
            #if particle[1] < rs and center_proj_sub[1] > self.length - rs:
            if particle[1] - center[1] < -self.length/2:
                offset[1] = -self.length
            #elif particle[1] > self.length - rs and center_proj_sub[1] < rs:
            elif particle[1] - center[1] > self.length/2:
                offset[1] = self.length
            
            #proj_x = particle[2] * self.tantheta + center[0]
            #print('\t{}'.format(particle[:3]))
            
            #d = shortest_distance_line_to_point(center_proj_height + offset, center_proj_sub+offset, particle)
            radii2 = rs**2
            
            
                
            # rotate axes by theta degrees about y-axis into new coordinate system so that z' axis is unimportant
            #rotx = (center_proj_height[0] - particle[0] + offset[0])*self.cos1theta + (center_proj_height[2] - particle[2])*self.sin1theta
            #d2 = rotx**2 + (center_proj_height[1] - particle[1] + offset[1])**2
            rotx = (center_float[0] - factor*self.length - particle[0] + offset[0])*self.cos1theta + (center_float[2] - particle[2])*self.sin1theta
            d2 = rotx**2 + (center_float[1] - particle[1] + offset[1])**2
            
            if self.verbose:
                print("Generated position: {}".format(center))
                print("x-coordinate in new coordinate system: {}".format(rotx))
                print("Offset: {}".format(offset))
                #print("center_proj_height + offset: {}".format(center_proj_height+offset))
                #print("center_proj_sub + offset: {}".format(center_proj_sub+offset))
                #print("Adjusted generated position: {}".format(center_proj_height))
                print("Existing particle position being considered: {}".format(particle[:3]))
                #print("Offset (if any): {}".format(offset))
                print("Minimum distance^2 between: {}".format(d2))
                print("Radii added together: {}".format(rs))
                
            if radii2 > d2:
                # it collides, calculate the new position
                rotz = np.sqrt(radii2 - d2)
                z = rotx*self.sin1theta + rotz*self.cos1theta + particle[2]
                x = rotx*self.cos1theta - rotz*self.sin1theta + particle[0]

                if z < radius:
                    # if it would collide with the surface below the substrate for this particle then we can skip this particle
                    continue
                if self.verbose:
                    print("Particles collide.")
                    print("Coordinates in new coordinate system: {}".format([rotx, center[1], rotz]))
                    print("Distance: {}".format(np.sqrt((x - particle[0])**2 + (center[1] - particle[1])**2 + (z - particle[2])**2)))
#                     print("Start: {}".format(center_proj_height))
#                     print("Start (reprojected): {}".format(center_proj_height + offset))
#                     print("Target: {}".format(center_proj_sub))
#                     print("Target (reprojected): {}".format(center_proj_sub+offset))
                    print("Collision position: {}".format([x, center[1], z]))
                    print("Colliding atom position: {}\n".format(particle[:3]))
                #print("Hit particle: {} -> {}".format(center, [x, center[1], z]))
                if profile:
                    pr.disable()
                    p = pstats.Stats(pr)
                    p.sort_stats('time').print_stats()
                    #pr.print_stats()
                return np.array([x, center[1], z]), particle
        x = adjusted_x - radius*self.tantheta
        #print("Didn't hit particle: {} -> {}".format(center, [x, center[1], radius]))
        if self.verbose:
            print("Particle did not collide with existing surface; landing at {}.\n".format([x, center[1], radius]))
        if profile:
            pr.disable()
            pr.print_stats()
        return np.array([x, center[1], radius]), -1
        
    def check_collision(self, center, radius, atoms):
        if(center[2] - radius < 0):
            return 2 # collision with substrate
        potential_collisions = self.get_nearby_particles(center, radius)
        if potential_collisions == []:
            return 0 # no nearby atoms
        for c in potential_collisions:
            if distance3D2PBC(atoms[c,:3], center, self.length) < (atoms[c,4] + radius)**2:
                return 3 + c # collision
        return 1 # no collision but atoms nearby
    
    def calc_priority(self, position):
        #dx = position[0] % self.bin_size
        return position[2]#-(dx * self.sin1theta - position[2] * self.cos1theta)

In [11]:
def follow_potential(pot_field, center, scale, L, h, distance, current_atom, lennardjones=True, deterministic=False, ratio=1.0):
    if distance == 0:
        return center
    if lennardjones:
        steps = int(round(scale*distance))
        position = center
        scaled_position = np.round(position * scale).astype(np.int32) % pot_field.shape[:3]
        
        if deterministic:
            remaining_distance = steps
            rads = int(round(scale*2*radius*ratio))
            while remaining_distance > 0:
                it = np.min([remaining_distance, rads])
                remaining_distance -= it
            
                minx = scaled_position[0]-it#steps
                if minx < 0:
                    minx = 0
                miny = scaled_position[1]-it#steps
                if miny < 0:
                    miny = 0
                minz = scaled_position[2]-it#steps
                if minz < int(np.ceil(scale*radius)):
                    minz = int(np.ceil(scale*radius))
                maxx = scaled_position[0]+it+1#steps+1
                if maxx > pot_field.shape[0]-1:
                    maxx = pot_field.shape[0]-1
                maxy = scaled_position[1]+it+1#steps+1
                if maxy > pot_field.shape[1]-1:
                    maxy = pot_field.shape[1]-1
                maxz = scaled_position[2]+it+1#steps+1

                region = pot_field[minx:maxx, miny:maxy, minz:maxz, 3]
                if 0 in region.shape:
                    print("steps: {}".format(steps))
                    print("real position: {}".format(position))
                    print("scaled_position: {}".format(scaled_position))
                    print("pot_field.shape: {}".format(pot_field.shape))
                    print("region.shape: {}".format(region.shape))
                    print("{}->{}, {}->{}, {}->{}".format(minx, maxx, miny, maxy, minz, maxz))
                minimum = np.argmin(region)
                minimum_i = np.unravel_index(minimum, region.shape)

                #print(pot_field[scaled_position[0], scaled_position[1], scaled_position[2], 3], minimum_i, region[minimum_i])
                if pot_field[scaled_position[0], scaled_position[1], scaled_position[2], 3] == region[minimum_i]:
                    return position
                minimum_i = np.array(minimum_i) + (minx, miny, minz)
            return np.array(minimum_i)/np.array((scale,scale,scale)) % (L,L,np.inf)
        
        for i in range(steps):
            try:
                direction = pot_field[scaled_position[0], scaled_position[1], scaled_position[2], :3]
            except IndexError:
                print(position, scaled_position)
                return position
            # check if all zero because we stop there
            if not np.any(direction):
                return position % (L,L,h)
            position = position + direction / scale
            if position[2] < 0:
                position[2] = 0
            elif position[2] > h:
                position[2] = h
            #print(direction, position, position % (L,L,h))
            position = position % (L,L,h)
            scaled_position = np.round(position * scale).astype(np.int32) % pot_field.shape[:3]
        return position % (L,L,h)
    else:
        k = 1
        diameter = 1
        position = center
        L2 = L**2
        step_size = 0.01
        for d in np.arange(0, distance, step_size):
            # get nearest neighbors within diameter radius
            displacementx = position[0] - atom_pos[:current_atom,0]
            displacementx = np.where(displacementx > L/2, -displacementx + L/2, displacementx)
            displacementx = np.where(displacementx < -L/2, -displacementx - L/2, displacementx)
            displacementy = position[1] - atom_pos[:current_atom,1]
            displacementy = np.where(displacementy > L/2, -displacementy + L/2, displacementy)
            displacementy = np.where(displacementy < -L/2, -displacementy - L/2, displacementy)
            displacementz = position[2] - atom_pos[:current_atom,2]
            distances = np.sqrt((displacementx)**2 + (displacementy)**2 + displacementz**2)
            idx = np.argwhere(np.where(distances < diameter, distances, 0)) # indices of nearest neighbor
            #print(d, distances.shape, distances[idx].shape)
            distances = distances[idx]
            displacement = np.hstack([displacementx[idx], displacementy[idx], displacementz[idx]])
            #print(displacement.shape)
            f = -np.sum(k * (distances - radius*2) * displacement/np.hstack([distances, distances, distances]), axis=0)
            if not ((f[0] == -0.0 and f[1] == -0.0 and f[2] == -0.0) or np.any(np.isnan(f))):
                position += step_size * f/np.linalg.norm(f)
                position = position % (L,L,np.inf)
            #print(position)
        #print(center)
        #print(position)
        #print(np.hstack([displacement, atom_pos[:,0][idx], atom_pos[:,1][idx], atom_pos[:,2][idx]]))
        return position % (L,L,np.inf)

In [135]:
# load an old simulation to get a BSP object ready
atoms, deposited, p = loadContinuum('STF_Si_L48_Th85_D0.45_N131072_1672876042' + '.npz')
for k in p[0]:
    print("{}: \t{}".format(k, p[0][k]))
pr = cProfile.Profile()
pr.enable()
bins = SlantedCorridors(p[0]['L'], 8, p[0]['theta'], 32, verbose=False)
for i in range(atoms.shape[0]):
    bins.add_to_bins(atoms[i,:3], p[0]['radius'], i)
pr.disable()
ps = pstats.Stats(pr)
ps.sort_stats('time').print_stats(10)

L: 	48
theta: 	85
phi: 	0
H: 	0.0
D: 	0.45
R: 	1
A: 	1
turns: 	0
stepper resolution: 	0
species: 	1
radius: 	0.15
weights: 	None
repetition: 	131072
spread: 	0
time: 	25748.940731048584
Total number of bins: 36
         4063242 function calls in 23.723 seconds

   Ordered by: internal time
   List reduced from 69 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1179648   11.765    0.000   11.765    0.000 <ipython-input-134-729a6894b696>:21(find_bin)
   131072    8.061    0.000   22.104    0.000 <ipython-input-134-729a6894b696>:43(find_bins)
  1048576    2.277    0.000    2.277    0.000 {built-in method numpy.array}
   131072    0.385    0.000    0.893    0.000 F:\Programs\Anaconda3_2020.11\lib\site-packages\sortedcontainers\sortedlist.py:1777(add)
        1    0.239    0.239   23.723   23.723 <ipython-input-135-83996588ce4f>:8(<module>)
   131072    0.216    0.000   23.484    0.000 <ipython-input-134-729a6894b696>:35(add_to_bins)


In [136]:
# generate random particle
rng = np.random.default_rng()
dests = np.hstack([rng.random(size=(100, 2), dtype=np.float32)*48, np.zeros((100,1), dtype=np.float32)+32]).squeeze()
#print("Generated particle position: {}".format(dest))

# drop test particle
#bins.verbose = False
#bins.find_bins(dest, p[0]['radius'])
#bins.find_bins(dest, radius)
radius = p[0]['radius']
#print(radius)
L = p[0]['L']
pr = cProfile.Profile()
pr.enable()
for i in range(dests.shape[0]):
    bins.drop_particle(dests[i], radius, atoms, False)
pr.disable()
ps = pstats.Stats(pr)
ps.sort_stats('time').print_stats(10)

         29305818 function calls (29029175 primitive calls) in 33.224 seconds

   Ordered by: internal time
   List reduced from 55 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      200    9.370    0.047   11.764    0.059 {built-in method builtins.sorted}
      100    9.285    0.093   32.461    0.325 <ipython-input-134-729a6894b696>:148(drop_particle)
 26214400    4.545    0.000    4.545    0.000 <ipython-input-134-729a6894b696>:152(<lambda>)
    13400    3.256    0.000    5.407    0.000 F:\Programs\Anaconda3_2020.11\lib\site-packages\sortedcontainers\sortedlist.py:1893(<genexpr>)
      100    2.915    0.029   20.966    0.210 F:\Programs\Anaconda3_2020.11\lib\site-packages\sortedcontainers\sortedset.py:664(update)
   276543    1.043    0.000    1.096    0.000 F:\Programs\Anaconda3_2020.11\lib\site-packages\sortedcontainers\sortedlist.py:600(_pos)
        1    0.763    0.763   33.224   33.224 <ipython-input-136-23b370a901b4>:15(

In [35]:
''' Slanted columns (no flux distribution) '''
# unit length is 1 nm
# diffusion remains to be solved
# need both a tree (to store roots/fiber IDs) and a spacial container (to improve collision detection)
# PBC
np.set_printoptions(precision=4)
np.seterr(all='raise')
reps = 32768
L = 32
theta = 85
phi = 0
turns = 0
species = 1
bin_size = 8
radius = 0.15
height = 32
D = 0.45
R = 1
A = 1

batch_size = 128
poses = [np.array([0,0,0]) for i in range(batch_size)]
cs = [-1 for i in range(batch_size)]



"""
    PROCESS:
        1) generate particle with transport angle
        2) send towards substrate
        3) during travel check that all particle centers in area are R1+R2 away at least
        4) once that isn't true a collision occurred: move the particle back until there isn't a collision
        5) diffuse the particle with a pseudo-Lennard Jones potential
        6) add particle to particle list, tree, and spacial container
        7) repeat a large number of times
        
    VISUALIZATION:
        1) It's a point list with float locations; use any point cloud viewer to view it
"""

# setup
atom_pos = np.zeros((reps, 7), dtype=np.float32) # x,y,z,sp,radius,fiber
bins = SlantedCorridors(L, bin_size, theta, height, verbose=False)
#print(bins.bins)
scale = 10
potentials = np.zeros((L*scale, L*scale, height*scale, 4), dtype=np.float32)
p = [{'L': L, 
    'theta': theta, 
    'phi': phi, 
    'H': height, 
    'D': D, 
    'R': R,
    'A': A,
    'turns': turns, 
    'stepper resolution': 0,
    'species': species,
    'radius': radius,
    'weights': None, 
    'repetition': reps, 
    'spread': 0}]

Vm = scale*(2*radius)
zero_pot = Vm / 2**(1/6)

if R != 1.0 and A != 1.0:
    print("Scaled:\tZero-potential point: {}\n\tMinimum-potential point: {}".format((R/A)**(1/6)*zero_pot, (2*R/A)**(1/6)*Vm))
    print("Real:\tZero-potential point: {}\n\tMinimum-potential point: {}".format((R/A)**(1/6)*zero_pot/scale, (2*R/A)**(1/6)*Vm/scale))
else:
    print("Scaled:\tZero-potential point: {}\n\tMinimum-potential point: {}".format(zero_pot, Vm))
    print("Real:\tZero-potential point: {}\n\tMinimum-potential point: {}".format(zero_pot/scale, Vm/scale))
diameter = 2 * scale
if diameter % 2 == 0:
    diameter += 1
template = np.zeros((diameter, diameter, diameter, 4), dtype=np.float32)
for i in range(diameter):
    ii = i - (diameter-1) // 2
    for j in range(diameter):
        jj = j - (diameter-1) // 2
        for k in range(diameter):
            kk = k - (diameter-1) // 2
            if ii == 0 and jj == 0 and kk == 0:
                template[i,j,k,:] = [0,0,0,np.inf]
            else:
                r = np.sqrt((ii)**2 + (jj)**2 + (kk)**2)
                magnitude = A*(zero_pot/r)**12 - R*(zero_pot/r)**6
                direction = np.array([ii,jj,kk])/r# * magnitude/np.abs(magnitude)
                template[i,j,k, 3] = magnitude
                template[i,j,k,:3] = direction
                if r > Vm:
                    template[i,j,k,:3] *= -1
                
rng = np.random.default_rng()
dests = np.hstack([rng.random(size=(reps, 2), dtype=np.float32)*L, np.zeros((reps,1), dtype=np.float32)+height])
#src = np.array([genh/tand(theta-90)*cosd(phi), genh/tand(theta-90)*sind(phi), 0])
#distance = distance3D(src, [0,0,0]) # distance from src to destination
#unit_vector = -src / distance # unit vector in travel direction
#print(src, distance, unit_vector)

update = max((int(reps // batch_size / 128), 1)) # frequency of printed updates
current_fiber_id = 0

# pr = cProfile.Profile()
# pr.enable()
# each particle must be treated in sequence
#print("Beginning simulation")
endsurf = 0
endpot = 0
endretreat = 0
endfly = 0
endsettle = 0
start = time.time()
dmax = 0
for i in range(reps//batch_size):
    #print("Particle {}".format(i+1))
    # get destination and source point
    for j in range(batch_size):
        startfly = time.time()
        dest = dests[i*batch_size + j]
        #shift = src + [dest[0], dest[1], dest[2]]

        #print("Traversing")
        startfly = time.time()
        if i < reps - 1:
            pos, cs[j] = bins.drop_particle(dest, radius, atom_pos)
        else:
            #pr = cProfile.Profile()
            #pr.enable()
            pos, cs[j] = bins.drop_particle(dest, radius, atom_pos, profile=False)
            #pr.disable()

        poses[j] = pos % [L,L,np.inf]

        if np.any(np.isnan(position)):
            print(position)
        endfly += time.time() - startfly
        #print("{} -> {}".format(dest, pos))

        # time to try diffusing
        #print("Following potential")
    if D > 0.0:
        for j in range(batch_size):
            startsurf = time.time()
            poses[j] = follow_potential(potentials, poses[j], scale, L, height, D, current_atom=i*batch_size + j, lennardjones=True, deterministic=True, ratio=1.0)
            endsurf += time.time() - startsurf
            #print("Adding potential")
            startpot = time.time()
            superimpose_potential(potentials, poses[j], scale, radius, template, diameter)
            endpot += time.time() - startpot

        #print("Settling atom")
    for j in range(batch_size):
        startsettle = time.time()
        if not np.any(np.isnan(poses[j])):
            atom_pos[i*batch_size+j,:3] = poses[j]
            #print("\tSettled at {}".format(position))
            atom_pos[i*batch_size+j,3] = species
            atom_pos[i*batch_size+j,4] = radius
            #if D > 0.0:
            #    superimpose_potential(potentials, poses[j], scale, radius, template, diameter)

    #         atom_pos[i+reps//2,:3] = dest / np.array([1,1,2])
    #         atom_pos[i+reps//2,3] = species
    #         atom_pos[i,4] = radius
    #         atom_pos[i+reps,:3] = dest / np.array([1,1,4])
    #         atom_pos[i+reps,3] = species
    #         atom_pos[i,4] = radius

            if cs[j] == -1:
                atom_pos[i*batch_size+j,5] = current_fiber_id
                current_fiber_id += 1
            else:
                atom_pos[i*batch_size+j,5] = atom_pos[cs[j], 5]
            #print(position, radius, i)
            bins.add_to_bins(poses[j], radius, i*batch_size+j)
        else:
            print(position)
        endsettle += time.time() - startsettle
    
        
    if i % update == update - 1:
        up_time = time.time() - start
        if i*batch_size+j+1 > up_time:
            print("{}/{} complete; {:.3f} seconds taken; {:.3f} reps per second.".format(i*batch_size+j+1, reps, up_time, (i*batch_size+j+1)/up_time))
        else:
            print("{}/{} complete; {:.3f} seconds taken; {:.3f} seconds per rep.".format(i*batch_size+j+1, reps, up_time, up_time/(i*batch_size+j+1)))
end = time.time() - start
if reps > end:
    print("{} reps completed in {} seconds;\n{} reps per second.".format(reps, end, reps/end))
else:
    print("{} reps completed in {} seconds;\n{} seconds per rep.".format(reps, end, end/reps))
print("{} seconds were traversal and collision.".format(endfly))
print("{} seconds were potential following.".format(endsurf))
print("{} seconds were potential addition.".format(endpot))
print("{} seconds were adding to collections.".format(endsettle))
for i in range(atom_pos.shape[0]):
    atom_pos[i, 6] = bins.find_bin_idx(atom_pos[i, :3])
#warnings.filterwarnings('default', category=RuntimeWarning)
print("Maximum diffusion distance: {}".format(dmax))

p[0]['time'] = end
#pr.disable()
filename = saveContinuum(atom_pos, p, "Si", zipped=True)
print(filename)
# pr.disable()
# ps = pstats.Stats(pr)
# ps.sort_stats('time').print_stats()

# snapshot = tracemalloc.take_snapshot()
# top_stats = snapshot.statistics('lineno')
# for stat in top_stats[:10]:
#     print(stat)
# pr.print_stats()

Total number of bins: 16
Scaled:	Zero-potential point: 2.672696154421018
	Minimum-potential point: 3.0
Real:	Zero-potential point: 0.2672696154421018
	Minimum-potential point: 0.3
256/32768 complete; 0.285 seconds taken; 898.044 reps per second.
512/32768 complete; 0.731 seconds taken; 700.243 reps per second.
768/32768 complete; 1.272 seconds taken; 603.633 reps per second.
1024/32768 complete; 1.844 seconds taken; 555.189 reps per second.
1280/32768 complete; 2.494 seconds taken; 513.267 reps per second.
1536/32768 complete; 3.144 seconds taken; 488.551 reps per second.
1792/32768 complete; 3.780 seconds taken; 474.053 reps per second.
2048/32768 complete; 4.462 seconds taken; 459.001 reps per second.
2304/32768 complete; 5.153 seconds taken; 447.082 reps per second.
2560/32768 complete; 5.871 seconds taken; 436.061 reps per second.
2816/32768 complete; 6.632 seconds taken; 424.619 reps per second.
3072/32768 complete; 7.414 seconds taken; 414.364 reps per second.
3328/32768 complete

30208/32768 complete; 188.511 seconds taken; 160.245 reps per second.
30464/32768 complete; 190.793 seconds taken; 159.671 reps per second.
30720/32768 complete; 193.074 seconds taken; 159.110 reps per second.
30976/32768 complete; 195.464 seconds taken; 158.474 reps per second.
31232/32768 complete; 197.871 seconds taken; 157.840 reps per second.
31488/32768 complete; 200.276 seconds taken; 157.223 reps per second.
31744/32768 complete; 202.812 seconds taken; 156.519 reps per second.
32000/32768 complete; 205.301 seconds taken; 155.868 reps per second.
32256/32768 complete; 207.694 seconds taken; 155.306 reps per second.
32512/32768 complete; 210.260 seconds taken; 154.628 reps per second.
32768/32768 complete; 212.803 seconds taken; 153.983 reps per second.
32768 reps completed in 212.80290007591248 seconds;
153.98286390040164 reps per second.
196.97659182548523 seconds were traversal and collision.
3.915703296661377 seconds were potential following.
6.584975719451904 seconds were po

In [17]:
#### tracemalloc.start()

# pr = cProfile.Profile()
# pr.enable()
#filename = 'structures\\continuum\\STF_Si_L32_Th85_D0.45_N65536_1673328957'
atom_pos, deposited, p = loadContinuum(filename + '.npz', folder = '')
reps2 = reps//8
atom_pos = np.vstack([atom_pos[:reps,:], np.zeros((reps2, atom_pos.shape[1]), dtype=np.float32)])
print(atom_pos.shape)
species = 2
radius = 0.15
height = 32

D = 3
p.append(p[0])
p[1]['repetition'] = reps2
p[1]['D'] = D
p[1]['species'] = species
p[1]['radius'] = radius

''' Do potentials twice, once for each species '''
potentials = np.zeros((L*scale, L*scale, height*scale, 4), dtype=np.float32)

for Vm in [scale*(2*.5), scale*(2*radius)]:
    R = 1.0
    A = 1.0
    zero_pot = Vm / 2**(1/6)
    if R != 1.0 or A != 1.0:
        print("Scaled:\tZero-potential point: {}\n\tMinimum-potential point: {}".format((R/A)**(1/6)*zero_pot, (2*R/A)**(1/6)*Vm))
        print("Real:\tZero-potential point: {}\n\tMinimum-potential point: {}".format((R/A)**(1/6)*zero_pot/scale, (2*R/A)**(1/6)*Vm/scale))
    else:
        print("Scaled:\tZero-potential point: {}\n\tMinimum-potential point: {}".format(zero_pot, Vm))
        print("Real:\tZero-potential point: {}\n\tMinimum-potential point: {}".format(zero_pot/scale, Vm/scale))
    diameter = 2 * scale
    if diameter % 2 == 0:
        diameter += 1
    template = np.zeros((diameter, diameter, diameter, 4), dtype=np.float32)
    for i in range(diameter):
        ii = i - (diameter-1) // 2
        for j in range(diameter):
            jj = j - (diameter-1) // 2
            for k in range(diameter):
                kk = k - (diameter-1) // 2
                if ii == 0 and jj == 0 and kk == 0:
                    template[i,j,k,:] = [0,0,0,np.inf]
                else:
                    r = np.sqrt((ii)**2 + (jj)**2 + (kk)**2)
                    magnitude = A*(zero_pot/r)**12 - R*(zero_pot/r)**6
                    direction = np.array([ii,jj,kk])/r# * magnitude/np.abs(magnitude)
                    template[i,j,k, 3] = magnitude
                    template[i,j,k,:3] = direction
                    if r > Vm:
                        template[i,j,k,:3] *= -1
                        
rng = np.random.default_rng()
dests = np.hstack([rng.random(size=(reps2, 2), dtype=np.float32)*L, np.zeros((reps2,1), dtype=np.float32)+height])

update = max((int(reps2 / 128), 1)) # frequency of printed updates
current_fiber_id = np.max(atom_pos[:,5] + 1)

# each particle must be treated in sequence
#print("Beginning simulation")
endsurf = 0
endpot = 0
endretreat = 0
endfly = 0
start = time.time()
dmax = 0
for i in range(reps, reps + reps2):
    #print("Particle {}".format(i+1))
    # get destination and source point
    startfly = time.time()
    dest = dests[i-reps]
    #shift = src + [dest[0], dest[1], dest[2]]
    
    #print("Traversing")
    
    pos, c = bins.drop_particle(dest, radius, atom_pos)
    
    position = pos % [L,L,np.inf]
    if np.any(np.isnan(position)):
        print(position)
    #print("{} -> {}".format(dest, pos))
    endfly += time.time() - startfly
    
    # time to try diffusing
    #print("Following potential")
    if D > 0.0:
        startsurf = time.time()
        position_new = follow_potential(potentials, position, scale, L, height, D/2, current_atom=i, lennardjones=True, deterministic=True, ratio=0.5)
        d = distance3DPBC(position, position_new, L)
        if d > dmax:
            dmax = d
        if d > D:
            print("Former position: {}".format(position))
            print("New position: {}".format(position_new))
            print("Distance: {}".format(d))
        position = position_new
        endsurf += time.time() - startsurf
        #print("Adding potential")
        startpot = time.time()
        superimpose_potential(potentials, position, scale, radius, template, diameter)
        endpot += time.time() - startpot
    
    #print("Settling atom")
    if not np.any(np.isnan(position)):
        atom_pos[i,:3] = position
        #print("\tSettled at {}".format(position))
        atom_pos[i,3] = species
        atom_pos[i,4] = radius
        
#         atom_pos[i+reps//2,:3] = dest / np.array([1,1,2])
#         atom_pos[i+reps//2,3] = species
#         atom_pos[i,4] = radius
#         atom_pos[i+reps,:3] = dest / np.array([1,1,4])
#         atom_pos[i+reps,3] = species
#         atom_pos[i,4] = radius
        
        if c == -1:
            atom_pos[i,5] = current_fiber_id
            current_fiber_id += 1
        else:
            atom_pos[i,5] = atom_pos[c, 5]
        #print(position, radius, i)
        bins.add_to_bins(position, radius, i)
    else:
        print(position)
    
        
    if i % update == update - 1:
        up_time = time.time() - start
        if i+1-reps > up_time:
            print("{}/{} complete; {:.3f} seconds taken; {:.3f} reps per second.".format(i+1-reps, reps2, up_time, (i+1-reps)/up_time))
        else:
            print("{}/{} complete; {:.3f} seconds taken; {:.3f} seconds per rep.".format(i+1-reps, reps, up_time, up_time/(i+1-reps)))
end = time.time() - start
if reps > end:
    print("{} reps completed in {} seconds;\n{} reps per second.".format(reps2, end, reps2/end))
else:
    print("{} reps completed in {} seconds;\n{} seconds per rep.".format(reps2, end, end/reps2))
print("{} seconds were traversal before collision.".format(endfly))
print("{} seconds were backing up from collision.".format(endretreat))
print("{} seconds were potential following.".format(endsurf))
print("{} seconds were potential addition.".format(endpot))
for i in range(atom_pos.shape[0]):
    atom_pos[i, 6] = bins.find_bin_idx(atom_pos[i, :3])
#warnings.filterwarnings('default', category=RuntimeWarning)
print("Maximum diffusion distance: {}".format(dmax))

p[1]['time'] = end
saveContinuum(atom_pos, p, "Si_Au", zipped=True)

# snapshot = tracemalloc.take_snapshot()
# top_stats = snapshot.statistics('lineno')
# for stat in top_stats[:10]:
#     print(stat)

# pr.print_stats()

(73728, 7)
Scaled:	Zero-potential point: 8.908987181403393
	Minimum-potential point: 10.0
Real:	Zero-potential point: 0.8908987181403394
	Minimum-potential point: 1.0
Scaled:	Zero-potential point: 2.672696154421018
	Minimum-potential point: 3.0
Real:	Zero-potential point: 0.2672696154421018
	Minimum-potential point: 0.3
64/8192 complete; 1.126 seconds taken; 56.825 reps per second.
128/8192 complete; 2.157 seconds taken; 59.328 reps per second.
192/8192 complete; 2.988 seconds taken; 64.264 reps per second.
256/8192 complete; 3.907 seconds taken; 65.525 reps per second.
320/8192 complete; 4.689 seconds taken; 68.244 reps per second.
384/8192 complete; 5.733 seconds taken; 66.977 reps per second.
448/8192 complete; 6.536 seconds taken; 68.538 reps per second.
512/8192 complete; 7.440 seconds taken; 68.820 reps per second.
576/8192 complete; 8.279 seconds taken; 69.578 reps per second.
640/8192 complete; 9.023 seconds taken; 70.932 reps per second.
704/8192 complete; 9.909 seconds taken;

7744/8192 complete; 92.936 seconds taken; 83.326 reps per second.
7808/8192 complete; 93.932 seconds taken; 83.124 reps per second.
7872/8192 complete; 94.698 seconds taken; 83.127 reps per second.
7936/8192 complete; 95.645 seconds taken; 82.974 reps per second.
8000/8192 complete; 96.468 seconds taken; 82.929 reps per second.
8064/8192 complete; 97.271 seconds taken; 82.902 reps per second.
8128/8192 complete; 98.054 seconds taken; 82.893 reps per second.
8192/8192 complete; 98.782 seconds taken; 82.930 reps per second.
8192 reps completed in 98.78324961662292 seconds;
82.92903940488992 reps per second.
92.09034204483032 seconds were traversal before collision.
0 seconds were backing up from collision.
3.3561787605285645 seconds were potential following.
1.8731229305267334 seconds were potential addition.
Maximum diffusion distance: 0.2529737420605582


'structures/continuum/STF_Si_Au_L32_Th85_D3_N73728_1673375548'